In [1]:
import geopandas as gpd
from sqlalchemy import create_engine

In [3]:
engine = create_engine(   
"mssql+pyodbc:///?odbc_connect="
"Driver%3D%7BODBC+Driver+17+for+SQL+Server%7D%3B"
"Server%3Drliddb.int.lcog.org%2C5433%3B"
"Database%3DRLIDGeo%3B"
"Trusted_Connection%3Dyes%3B"
"ApplicationIntent%3DReadWrite%3B"
"WSID%3Dclwrk4087.int.lcog.org%3B")

In [37]:
def readLTDstops():
    sql = '''
    SELECT 
    stop_number AS stopNumber,
    stop_name AS stopName,
    longitude,
    latitude,
    Shape.STAsBinary() AS geom
    FROM dbo.LTD_Stop;
    '''
    
    ltdstops = gpd.GeoDataFrame.from_postgis(sql, engine, geom_col='geom')
    ltdstops.crs = "EPSG:2914"
    
    #ltdstops = ltdstops.to_crs(epsg=3857)
    return ltdstops

In [4]:
LTDstops = readLTDstops()

In [5]:
import pandas as pd

In [9]:
bikes_on_buses = pd.read_excel(r"T:\Data\LTD Data\BikeOnBuses\Monthly\LTD Bike Count_2019.xlsx")

In [11]:
bikes_on_buses.head()

,srv,date,block,trip_end,time,route,dir,stop,stop_name,latitude,longitude,bus,odometer,desc,qty
0,wkd,2019-01-02,2,05:13:00,05:09:00,11,O,14,W/S of 69th N of Main St,44.046193,-122.903940,6204,8.39,bike on rack,1
1,wkd,2019-01-02,2,05:31:00,05:21:00,11,I,44,N/S of Main E of 32nd,44.045906,-122.979896,6204,12.21,bike on rack,1
2,wkd,2019-01-02,2,07:39:00,07:32:00,18,I,58,W/S of Mohawk N of G,44.053969,-123.004102,6204,40.09,bike on rack,1
3,wkd,2019-01-02,2,07:57:00,07:41:00,18,O,2503,"Springfield Station, Bay C",44.045114,-123.021270,6204,41.61,bike on rack,1
4,wkd,2019-01-02,3,06:57:00,06:22:00,12,O,2119,"Eugene Station, Bay S",44.048689,-123.093716,1112,46.52,bike on rack,3


In [10]:
bikes_on_buses.route.unique()

array([11, 18, 12, 40, 41, 24, 93, 51, 96, 55, 28, 52, 36, 67, 66, 91, 27,
       95, 13, 78, 85, 82, 98, 1, 17, 104, 105, 103, 92, 33, 73, 81,
       '79x'], dtype=object)

In [17]:
Emx_Routes = [101, 102, 103, 104, 105]

In [25]:
Emx_Stops = bikes_on_buses[bikes_on_buses.route.isin(Emx_Routes)].stop.unique()

In [28]:
type(Emx_Stops[0])

int

In [31]:
Emx_Stops = ["0" + str(Emx_Stop) for Emx_Stop in Emx_Stops]

In [32]:
LTDstops.columns

Index(['stopNumber', 'stopName', 'longitude', 'latitude', 'geom'], dtype='object')

In [33]:
LTD_Emx_stops = LTDstops[LTDstops.stopNumber.isin(Emx_Stops)]

In [34]:
LTD_Emx_stops.to_file(r"T:\DCProjects\Support\EUG\MiddleHousing\EmX_stops.shp", index=False)

In [35]:
LTD_stops = LTDstops[~LTDstops.stopNumber.isin(Emx_Stops)]

In [36]:
LTD_Emx_stops.to_file(r"T:\DCProjects\Support\EUG\MiddleHousing\LTD_stops.shp", index=False)